# **FITS to HDF5**

In [19]:
import numpy as np
import pandas as pd
import h5py
import astropy.table
from astropy.table import Table
import astropy.io.fits as pf

In [13]:
filename = '/home/raulteixeira/scratch-midway2/CosmicShearData/shear_test_coadd_object_gold.fits'
pf.open(filename).info()

Filename: /home/raulteixeira/scratch-midway2/CosmicShearData/shear_test_coadd_object_gold.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    238   18617785R x 45C   [K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, J, J, J, J, I, I, I, I, K, J, J, J, J, 48A]   


In [21]:
data = Table.read('/home/raulteixeira/scratch-midway2/CosmicShearData/shear_test_coadd_object_gold.fits')

In [23]:
#s/g cut
WAVG_SPREAD_MODEL_I = data['WAVG_SPREAD_MODEL_I']
WAVG_SPREADERR_MODEL_I = data['WAVG_SPREADERR_MODEL_I']

#conditions
c1 = (WAVG_SPREAD_MODEL_I+3*WAVG_SPREADERR_MODEL_I<=0.005)
c2 = (WAVG_SPREAD_MODEL_I+WAVG_SPREADERR_MODEL_I<=0.003)
c3 = (WAVG_SPREAD_MODEL_I-WAVG_SPREADERR_MODEL_I<=0.002)


#conditions
nc1 = np.logical_not(c1)
nc2 = np.logical_not(c2)
nc3 = np.logical_not(c3)

sc1 = np.logical_and(np.logical_and(c1, c2), c3)
sc2 = np.logical_and(np.logical_and(nc1, c2), c3)
sc3 = np.logical_and(np.logical_and(c1, nc2), c3)
sc4 = np.logical_and(np.logical_and(c1, c2), nc3)

hc1 = np.logical_or(sc1, sc2)
hc2 = np.logical_or(sc3, sc4)

sg_mask = np.logical_or(hc1, hc2)

In [ ]:
#s/g cut Y6



In [24]:
data = data[sg_mask]

In [25]:
# creating mask
mask1 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)
mask2 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) 
mask3 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_Z']!=99)
mask4 = (data['MAG_AUTO_G']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)
mask5 = (data['MAG_AUTO_R']!=99) * (data['MAG_AUTO_I']!=99) * (data['MAG_AUTO_Z']!=99)

mask = mask1 + mask2 + mask3 + mask4 + mask5

In [26]:
data=data[mask]

In [27]:
bandlist = ['G', 'R', 'I', 'Z']
for band in bandlist:
    data[f'MAG_AUTO_{band}'][data[f'MAG_AUTO_{band}']==99]=-99
    data[f'MAGERR_AUTO_{band}'][data[f'MAG_AUTO_{band}']==99]=0

In [28]:
tiles = np.unique(np.array(data['TILENAME']))

In [31]:
#creating the table to be converted
for tile in np.unique(data['TILENAME']):
    #converting to pandas dataframe
    masktile=data['TILENAME']==tile
    table_i = data[masktile]['COADD_OBJECT_ID', 'TILENAME', 'MAG_AUTO_G', 'MAG_AUTO_R'\
                         , 'MAG_AUTO_I', 'MAG_AUTO_Z', 'MAGERR_AUTO_G', 'MAGERR_AUTO_R'\
                         , 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z'] 
                         #[tab_all['extended_class']>1]
    dframe_i = pd.DataFrame(data=table_i['COADD_OBJECT_ID']\
                            , columns = ['COADD_OBJECT_ID'])
    for label in table_i.columns:
            dframe_i[label] = data[label][masktile]
    dframe_i.to_hdf(f'/home/raulteixeira/scratch-midway2/CosmicShearData/bpztiles/pzinput/pzinput_sg_{tile}.h5', key='df')

KeyboardInterrupt: 

In [ ]:
tile = tiles[0].astype('str')

In [14]:
#converting to pandas dataframe
masktile=data['TILENAME']==tile
table_i = data[mask][masktile]['COADD_OBJECT_ID', 'TILENAME', 'MAG_AUTO_G', 'MAG_AUTO_R'\
                     , 'MAG_AUTO_I', 'MAG_AUTO_Z', 'MAGERR_AUTO_G', 'MAGERR_AUTO_R'\
                     , 'MAGERR_AUTO_I', 'MAGERR_AUTO_Z'] 
                     #[tab_all['extended_class']>1]
dframe_i = pd.DataFrame(data=table_i['COADD_OBJECT_ID']\
                        , columns = ['COADD_OBJECT_ID'])
for label in table_i.columns:
        dframe_i[label] = data[label][masktile]
dframe_i.to_hdf('/home/raulteixeira/photoz/CosmicShearData/bpztiles/pzinput/pz_input_gold_'+tile+'.h5', key='df')